In [ ]:
import pandas as pd
import surprise as sp
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import KFold
from surprise import accuracy
#from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
df = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/suprise_df.csv")
df5 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/suprise_5.csv")
dfb = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/suprise_b.csv")
df_weight = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/suprise_weight.csv", index_col=0)

In [ ]:
top_10 = df_weight.groupby('user').apply(lambda x: x.nlargest(10, 'ranking_weight')).droplevel(level=0)

In [ ]:
new = top_10.drop(columns='ranking_weight')

In [ ]:
new

In [ ]:
list_int = [1,2,3,4,5,6,7,8,9,10]
top_10['rank'] = np.tile(list_int, len(top_10)//len(list_int))

In [ ]:
df_non = df.drop(df[df.city == 'None'].index)



In [ ]:
df_weight['ranking_weight'].describe()


In [ ]:
df_weight.sort_values('ranking_weight', inplace=True)

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2)
df5_train, df5_test = train_test_split(df5, test_size=0.2)
dfb_train, dfb_test = train_test_split(dfb, test_size=0.2)
weight_train, weight_test = train_test_split(df_weight, test_size=0.2)

#e_train, e_test = train_test_split(eight, test_size=0.2)



In [ ]:
df_weight

In [ ]:
# Change it back to countries
#df.iloc[:, 1] = labelencoder.inverse_transform(df.iloc[:, 1])
labelencoder = LabelEncoder()
#df.iloc[:, 1] = labelencoder.fit_transform(df.iloc[:, 1])
df

In [ ]:
df.city.value_counts()

In [ ]:
df_weight.sort_values('ranking_weight')

In [ ]:
top_10.max()

In [ ]:
readere = Reader(rating_scale=(0.249561, 2.221475))
datawe = sp.Dataset.load_from_df(df_weight[['user', 'city', 'ranking_weight']], readere)

readerten = Reader(rating_scale=(1, 10))
dataten = sp.Dataset.load_from_df(new[['user', 'city', 'rank']], readerten)

In [ ]:
reader= Reader(rating_scale=(1.0, 3.0))
data = sp.Dataset.load_from_df(df_non[['user_id', 'city', 'rank']], reader)

reader5= Reader(rating_scale=(1.0, 5.0))
data5 = sp.Dataset.load_from_df(df5[['user_id', 'city', 'rank']], reader5)

readerb = Reader(rating_scale=(1.0, 5.0))
datab = sp.Dataset.load_from_df(dfb[['user_id', 'city', 'rank']], readerb)

In [ ]:
# Using surprise train/test
#trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
benchmark = []
# Iterate over all algorithms
for algorithm in [sp.SVD(), sp.NMF(), sp.NormalPredictor(), sp.KNNBasic(), sp.KNNWithZScore(), sp.CoClustering(),
                 sp.SlopeOne(), sp.KNNBaseline(), sp.KNNWithMeans(), sp.SVDpp(), sp.BaselineOnly()]:
# Perform cross validation
    results = cross_validate(algorithm, dataten, measures=['RMSE'], cv=3, verbose=False)
# Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]],index=['Algorithm']))
    benchmark.append(tmp)

In [ ]:
benchmark = pd.DataFrame(benchmark)
benchmark

### GridSearch CV 

With SVD

In [ ]:
param_grid = {'n_factors': [25, 30, 35, 40, 100], 'n_epochs': [15, 20, 25], 'lr_all': [0.001, 0.003, 0.005, 0.008],               'reg_all': [0.08, 0.1, 0.15, 0.02]}
gs = GridSearchCV(sp.SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(dataw) 
algo = gs.best_estimator['rmse']
print(gs.best_score['rmse']) 
print(gs.best_params['rmse'])

In [ ]:

model = sp.SVD(n_factors=100, n_epochs=25, lr_all=.001, reg_all=.08)
trainset = data.build_full_trainset()
model.fit(trainset)

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = model.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
algo2 = sp.SVD(n_factors= 25, n_epochs = 15, lr_all = .001, reg_all = .08)
kf = KFold(n_splits=3)


for trainset, testset in kf.split(dataw):

    # train and test algorithm.
    algo2.fit(trainset)
    predictions = algo2.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions

In [ ]:
sim = {'name': 'pearson_baseline', 'user_based': True, 'min_support': 5}
algo = sp.KNNBasic(k_min = 2, sim_options = sim)
kf = KFold(n_splits=3)

for trainset, testset in kf.split(datawe):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
pd.DataFrame(predictions)

In [ ]:
algo3 = sp.BaselineOnly()
kf = KFold(n_splits=3)


for trainset, testset in kf.split(dataw):

    # train and test algorithm.
    algo3.fit(trainset)
    predictions = algo3.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
trainset = dataten.build_full_trainset()
algo = sp.CoClustering()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=5)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
top_10[top_10['city'] == 'Jerusalem']

In [ ]:
data

In [ ]:
df[df['city'] == 'Zurich']

In [ ]:
from collections import defaultdict

from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=.01):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


kf = KFold(n_splits=5)
algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))